In [1]:
# step1. 라이브러리 불러오기
import os
import json
from typing import Dict, List, Any
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv

load_dotenv()
print("✅ 라이브러리 불러오기 완료!")

✅ 라이브러리 불러오기 완료!


In [2]:
# step2. LLM 초기화 및 설정
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3) 
    print("✅ ChatOpenAI LLM 초기화 완료!")
except Exception as e:
    llm = None
    
# JSON 출력 구조 정의
RECOMMENDATION_OUTPUT_SCHEMA = {
    "recommendation_id": "int (1부터 시작)",
    "recommendation_title": "string (개선 항목의 제목)",
    "mitigation_step": "string (실행 가능한 구체적인 개선 조치 설명)",
    "priority": "string (High, Medium, Low 중 하나)",
    "effort_level": "string (Low, Medium, High 중 하나)",
    "relevant_standard": "string (EU AI Act, OECD, UNESCO 중 가장 관련 높은 기준 명시)"
}

✅ ChatOpenAI LLM 초기화 완료!


In [3]:
# step3. 개선 권고 프롬프트 정의
RECOMMENDATION_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "당신은 AI 윤리 리스크 개선 전문가입니다. "
         "제공된 리스크 평가 결과와 '개선 권고 초점'을 기반으로, "
         "구체적이고 실행 가능한 개선 권고안을 3가지 이상 생성하여 JSON 배열 형식으로만 반환하세요. "
         "반드시 출력 JSON 스키마를 준수해야 합니다."
         "\n\n[출력 JSON 스키마]:\n{schema}"
        ),
        ("human", 
         "--- 서비스 및 리스크 평가 정보 ---"
         "\n서비스명: {service_name}"
         "\n평가 리스크 카테고리: {category}"
         "\n위험도 수준: {risk_level}"
         "\n평가 요약: {assessment_summary}"
         "\n--- 개선 권고 초점 ---"
         "\n{recommendation_focus}"
         "\n\n위 정보를 바탕으로 **{recommendation_focus}**를 해결할 수 있는 구체적인 개선 권고안을 생성하세요. "
         "우선순위(Priority)는 리스크 수준과 연관시켜 설정하세요."
        )
    ]
)

In [4]:
# step4. 권고안 생성 함수 정의 (State를 입력으로 받도록 변경)
def generate_recommendations_and_update_state(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    State에서 평가 결과를 읽고, 개선 권고안을 생성한 후, State에 결과를 저장합니다.
    """
    if not llm:
        state['recommendation_status'] = "Error: LLM failed to initialize."
        return state
    
    assessment_data = state.get('assessment_result', {})
    if not assessment_data or not assessment_data.get('assessed_risks'):
        state['recommendation_status'] = "Error: Missing assessment_result in state."
        return state
    
    service_name = assessment_data.get('service_name', 'AI 서비스')
    risks_to_mitigate = [
        r for r in assessment_data['assessed_risks'] 
        if r['risk_level'] in ['High', 'Limited'] # High와 Limited 리스크만 권고안 생성
    ]
    
    all_recommendations = []
    parser = JsonOutputParser(pydantic_object=None)
    
    print(f"\n💡 Mitigation Recommender 시작: {service_name}의 개선 권고안 생성")
    
    for risk in risks_to_mitigate:
        category = risk['category']
        print(f"\n   ⚙️ {category.upper()} 리스크 개선 권고안 생성 중...")
        
        chain = RECOMMENDATION_PROMPT | llm | parser

        try:
            recommendation_list = chain.invoke({
                "schema": json.dumps(RECOMMENDATION_OUTPUT_SCHEMA, indent=2, ensure_ascii=False),
                "service_name": service_name,
                "category": category,
                "risk_level": risk['risk_level'],
                "assessment_summary": risk['assessment_summary'],
                "recommendation_focus": risk['recommendation_focus']
            })
            
            # 카테고리 정보와 함께 리스트에 추가
            for rec in recommendation_list:
                rec['risk_category'] = category
            
            all_recommendations.extend(recommendation_list)
            print(f"     ✅ 권고안 {len(recommendation_list)}개 생성 완료: {category.upper()}")
            
        except Exception as e:
            print(f"     ❌ 권고안 생성 실패 ({category}): {e}")
            
    # 💡 State에 권고안 결과 저장
    state['recommendation_result'] = all_recommendations
    state['recommendation_status'] = "Success"
    
    print("\n✅ Mitigation Recommender 완료 및 State 업데이트 완료!")
    return state

In [5]:
# step5. 테스트 실행 (Risk Assessor의 결과 더미 데이터 사용)
# 이 데이터는 실제로는 state['assessment_result']에 저장되어 있어야 함.

test_state = {
    'assessment_result': {
        "service_name": "이력서 분석 추천 시스템",
        "assessed_risks": [
            {
                "category": "bias",
                "risk_level": "High",
                "assessment_summary": "채용 AI는 고위험으로 분류되며, 최신 사회 이슈에서 성별/출신 편향성 문제가 명확히 드러남. 즉각적인 조치가 필요함.",
                "recommendation_focus": "데이터 다양성 확보 및 모델 공정성 감사 프로세스 구축"
            },
            {
                "category": "privacy",
                "risk_level": "Limited",
                "assessment_summary": "민감 정보를 처리하나, 익명화 기술 적용이 가능하여 위험도는 중간 수준임. 데이터 통제권 강화가 필요함.",
                "recommendation_focus": "익명화 수준 강화 및 데이터 주체 동의 명확화"
            },
            {
                "category": "transparency",
                "risk_level": "Minimal",
                "assessment_summary": "알고리즘의 결정 과정 공개 의무는 있으나, 현재까지 큰 사회적 이슈는 없음. 설명 가능성 문서화 수준으로 충분함.",
                "recommendation_focus": "결정 근거 문서화 및 이해관계자에게 설명 가능한 인터페이스 제공"
            }
        ]
    }
}

print("\n" + "="*60)
print("💡 Mitigation Recommender 시작 (State 기반 테스트)...")
print("="*60)

updated_state = generate_recommendations_and_update_state(test_state)

print("\n" + "="*60)
print("📢 Mitigation Recommender 최종 결과 (업데이트된 State 내용)")
print("="*60)
print(f"총 생성된 권고안 개수: {len(updated_state.get('recommendation_result', []))}개")
print(json.dumps(updated_state.get('recommendation_result'), indent=2, ensure_ascii=False))

print(f"\n🔗 다음 단계: Report Composer는 state['assessment_result']와 state['recommendation_result']를 입력으로 사용합니다.")
print("="*60)


💡 Mitigation Recommender 시작 (State 기반 테스트)...

💡 Mitigation Recommender 시작: 이력서 분석 추천 시스템의 개선 권고안 생성

   ⚙️ BIAS 리스크 개선 권고안 생성 중...
     ✅ 권고안 3개 생성 완료: BIAS

   ⚙️ PRIVACY 리스크 개선 권고안 생성 중...
     ✅ 권고안 3개 생성 완료: PRIVACY

✅ Mitigation Recommender 완료 및 State 업데이트 완료!

📢 Mitigation Recommender 최종 결과 (업데이트된 State 내용)
총 생성된 권고안 개수: 6개
[
  {
    "recommendation_id": 1,
    "recommendation_title": "데이터 수집 프로세스 개선",
    "mitigation_step": "다양한 인구 통계학적 배경을 반영한 데이터 수집을 위해, 채용 시장의 다양한 집단을 포함하는 샘플링 전략을 수립하고, 특정 성별 및 출신 배경에 대한 편향을 최소화하기 위한 가이드라인을 마련한다.",
    "priority": "High",
    "effort_level": "Medium",
    "relevant_standard": "EU AI Act",
    "risk_category": "bias"
  },
  {
    "recommendation_id": 2,
    "recommendation_title": "모델 공정성 감사 프로세스 구축",
    "mitigation_step": "AI 모델의 공정성을 평가하기 위한 독립적인 감사 팀을 구성하고, 정기적으로 모델의 성능을 분석하여 성별 및 출신에 따른 편향성을 점검하는 프로세스를 수립한다.",
    "priority": "High",
    "effort_level": "High",
    "relevant_standard": "OECD",
    "risk_category": "bias"
  },
  {
    "r